# 02 - Labeling Intent

---

### Intent Categories

| Intent Label        | Description                                      |
|---------------------|------------------------------------------------|
| `cancel_service`    | Customer wants to cancel account or switch plan|
| `billing_issue`     | Complaints or questions about billing           |
| `technical_issue`   | Problems with device, network, or service       |
| `account_help`      | Issues with account management or login         |
| `upgrade_request`   | Requests to upgrade device or plan               |
| `general_question`  | General questions or inquiries                    |
| `positive_feedback` | Praise or compliments                             |
| `complaint`         | Negative feedback not related to billing or tech|
| `other`             | Anything else or unclear                          |

# Labeling and Logic of Categorization

In [1]:
import pandas as pd
import sys
import os


# Fixing routing issue
project_root = os.path.abspath('..')
sys.path.append(project_root)

from scripts.preprocess import assign_intent

# Loads Data from Cleaned CSV
df = pd.read_csv("../data/processed/cleaned_tweets.csv")

# Sample 250 rows to label
golden_df = df.sample(250, random_state=42).copy()

# Creates Categories to put into column later
intent_categories = [
    "Billing",
    "Technical Support",
    "Account Management",
    "Complaint",
    "Praise/Thank You",
    "Other"
]

keywords = {
    "Billing": ["bill", "charge", "payment", "refund"],
    "Technical Support": ["error", "issue", "problem", "disconnect", "slow"],
    "Account Management": ["password", "login", "account", "reset"],
    "Complaint": ["bad", "terrible", "worst", "disappointed", "angry"],
    "Praise/Thank You": ["thank", "great", "love", "awesome", "appreciate"]
}

# Apply Categorization Function to Datarframe and Write to CSV file

In [2]:
df['intent'] = df['cleaned_text'].apply(lambda text: assign_intent(text, keywords))
print(df[['cleaned_text', 'intent']].sample(10))

df['tweet_length'] = df['cleaned_text'].str.len()

print(df[['cleaned_text', 'tweet_length']].sample(10))
print(df[['cleaned_text', 'sentiment_score']].sample(10))

# Writes golden dataset to csv file
df.to_csv("../data/processed/golden_intent_labeled.csv", index=False)
print("Golden dataset saved")

                                           cleaned_text             intent
6741  hi your team is not helping me in app desperat...              Other
1421                            rip i forgot about that              Other
5171                                            yea bye              Other
5548   youre welcome george meet us in dm whenever y...   Praise/Thank You
4590   no worries please follow us via twitter and l...   Praise/Thank You
959    hi what type of pad are you using and where w...              Other
7810   hey there try resetting your password here if...  Technical Support
9771  yes when i mentioned xfinity he knew who it wa...              Other
780    i would recommend signing up for our free ser...              Other
5124   theres no fresh just frozen the only in are o...              Other
                                           cleaned_text  tweet_length
7730   well if that dont beat all much obliged a cus...            60
8102   i am sorry for the poor expe

# Quick Evaluation

In [3]:
# Peep the examples by intent
for label in df['intent'].unique():
    print(f"Intent: {label}")
    sample = df[df['intent'] == label][['cleaned_text']].dropna().sample(3, random_state=1)
    for i, row in enumerate(sample.itertuples(index=False), 1):
        print(f"\n  {i}. {row.cleaned_text}")

Intent: Other

  1.  its a yes or no question

  2.  ive restarted my phone several times still doesnt work

  3.  why does my personal station keep playing song i expressly marked dislike 
Intent: Praise/Thank You

  1. the state of my fire emblem warriors special edition thanks unacceptable 

  2. thanks to and for getting my retrochallenge project parts here a day early 

  3.  thank you
Intent: Complaint

  1. dear excited re low fare finder disappointed but no surprise that only search options are from london northsouthdivide

  2. dont let fool you they dont they just wasted almost full hours of my time tonight and the cs rep knew she couldnt do what i needed terriblecustomerservice xfinity when can we get 

  3.  yes you need to contact spectrumbad amp cancel your acct always rated last in svcereliability would love to hear from you for better deals 
Intent: Account Management

  1.  hi deanna i do not have access to your account via twitter please reach out to us here for more 

# Additional Features